In [1]:
%pip install GitPython

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_csv(r"c:\Users\HW5584\Downloads\Devops Migration Overview.csv")

In [3]:
df.head()

,Reponame,old organisation,old project,new project,another refresh needed,freeze date,day 1 port needed,old URL,new URL
0,ElectroMagneticProcessing,Lithography,Analysis,10_BG_Analysis,x,21.11.2024,NaN,https://dev.azure.com/Lithography/Analysis/_gi...,https://ZEISSgroup-SMT@dev.azure.com/ZEISSgrou...
1,arcas_au3400,Lithography,AU3400,10_BG_AU3400,x,01.12.2024,NaN,https://dev.azure.com/Lithography/AU3400/_git/...,https://dev.azure.com/ZEISSgroup-SMT/10_BG_AU3...
2,astrea_au3400,Lithography,AU3400,10_BG_AU3400,x,01.12.2024,NaN,https://dev.azure.com/Lithography/AU3400/_git/...,https://dev.azure.com/ZEISSgroup-SMT/10_BG_AU3...
3,arcas_au5000,Beyond-Gravity,BGI AU5000,10_BG_AU5000,x,01.12.2024,NaN,https://dev.azure.com/Beyond-Gravity/BGI AU500...,https://dev.azure.com/ZEISSgroup-SMT/10_BG_AU5...
4,Astrea_AU5000,Beyond-Gravity,BGI AU5000,10_BG_AU5000,x,01.12.2024,NaN,https://dev.azure.com/Beyond-Gravity/BGI AU500...,https://dev.azure.com/ZEISSgroup-SMT/10_BG_AU5...


In [4]:
token = r"49QzGd8ZOLTWdoMc0S3M0cZkVVsBMTua01tUMYOkTUnEwxebgYdhJQQJ99AKACAAAAAHsyrdAAASAZDOULjm"

In [4]:
df = df[df["old URL"] == "https://dev.azure.com/Beyond-Gravity/BGI AU5000/_git/Astrea_AU5000"]

In [ ]:
git@ssh.dev.azure.com:v3/Beyond-Gravity/BGI%20AU5000/Astrea_AU5000

In [ ]:
import os
import git

# Directory to clone repositories into
clone_directory = r"C:\Users\HW5584\git\clones"
# Create a directory to hold cloned repositories
os.makedirs(clone_directory, exist_ok=True)

results = []
for idx, line in df.iterrows():

    repo_url = line["old URL"]
    
    repo_url = f'{repo_url.split("//")[0]}//{token}@{repo_url.split("//")[1]}'
    repo_url = repo_url.replace(" ", "%20")
    
    remote_url = line["new URL"]
    remote_url = remote_url.replace(" ", "%20")
    
    # Extract the repo name from the URL
    repo_name = repo_url.split('/')[-1].replace('.git', '')
    repo_path = os.path.join(clone_directory, repo_name)
    pushed_branches = []

    try:
        # Clone the repository
        print(f"Cloning {repo_url}...")
        if os.path.exists(repo_path):
            print(f"Repository {repo_name} already exists. Fetching and pulling latest changes...")
            repo = git.Repo(repo_path)
            repo.git.fetch()  # Fetch the latest changes
            # repo.git.pull()   # Pull the latest changes
        else:
            git.Repo.clone_from(repo_url, repo_path)
            # Change to the cloned repository directory
            repo = git.Repo(repo_path)
        
        print("Available branches:")
        print(repo.git.branch('-a'))  # List all branches

        # Add the new remote
        print(f"Adding new remote: {remote_url}...")
        remote_name = "new_remote"
        if remote_name not in [remote.name for remote in repo.remotes]:
            print(f"Adding new remote: {remote_url}...")
            repo.create_remote(remote_name, remote_url)
        else:
            print(f"Remote '{remote_name}' already exists. Skipping creation.")
        print(list([remote.name for remote in repo.remotes]))

        # Push all branches to the new remote
        print(f"Pushing all branches to new remote...")
        for branch in repo.references:
            try:
                print(f"Pushing branch: {branch.name}")
                repo.git.checkout(branch.name)
                repo.git.pull("origin", branch.name.lstrip("origin/"))
                repo.git.push(remote_name, branch.name)
                pushed_branches.append(branch.name.lstrip("origin/"))
            except Exception as e:
                print(f"Cannot push branch {branch.name}, error: {e}")
                continue
        
        print(f"Pushing all tags to new remote...")
        repo.git.push(remote_name, '--tags')

        print(f"Successfully cloned and pushed {repo_name} to {remote_url}.\n\n")
        results.append({"repo_name": repo_name, "status": "Success", "error": None, "cloned_branches": pushed_branches})

    except Exception as e:
        print(f"An error occurred with {repo_name}: {e}\n\n")
        results.append({"repo_name": repo_name, "status": "Failed", "error": str(e), "cloned_branches": pushed_branches})

print("All operations completed.")
df_results = pd.DataFrame(results)

# Display the DataFrame
print("\nResults:")
print(df_results)

# Optionally, save the DataFrame to a CSV file
df_results.to_csv("repository_processing_results_2.0.csv", index=False)


Cloning https://49QzGd8ZOLTWdoMc0S3M0cZkVVsBMTua01tUMYOkTUnEwxebgYdhJQQJ99AKACAAAAAHsyrdAAASAZDOULjm@dev.azure.com/Lithography/Analysis/_git/ElectroMagneticProcessing...
An error occurred with ElectroMagneticProcessing: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://*****@dev.azure.com/Lithography/Analysis/_git/ElectroMagneticProcessing C:\Users\HW5584\git\clones\ElectroMagneticProcessing
  stderr: 'Cloning into 'C:\Users\HW5584\git\clones\ElectroMagneticProcessing'...
remote: TF401019: The Git repository with name or identifier ElectroMagneticProcessing does not exist or you do not have permissions for the operation you are attempting.
fatal: repository 'https://dev.azure.com/Lithography/Analysis/_git/ElectroMagneticProcessing/' not found
'


Cloning https://49QzGd8ZOLTWdoMc0S3M0cZkVVsBMTua01tUMYOkTUnEwxebgYdhJQQJ99AKACAAAAAHsyrdAAASAZDOULjm@dev.azure.com/Lithography/AU3400/_git/arcas_au3400...
Repository arcas_au3400 already exists. Fetching and pulling l

In [14]:
df_results[df_results["status"] == "Success"].to_csv("Transfered_repos.csv")